# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
import csv
cities_df = "../starter_code/city_data_reports.csv"
cities_data_df = pd.read_csv(cities_df)
cities_data_df

,Unnamed: 0,City,City Latitude,City Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),City Country,City Datetime
0,0,port blair,11.67,92.75,84.07,84.07,74.0,4.0,8.28,IN,1.604202e+09
1,1,kakonko,-3.28,30.96,61.97,61.97,85.0,50.0,0.89,TZ,1.604202e+09
2,2,saint george,37.10,-113.58,59.09,60.80,17.0,1.0,3.36,US,1.604202e+09
3,3,rikitea,-23.12,-134.97,75.90,75.90,72.0,0.0,9.44,PF,1.604202e+09
4,4,punta arenas,-53.15,-70.92,44.60,44.60,87.0,0.0,5.82,CL,1.604202e+09
...,...,...,...,...,...,...,...,...,...,...,...
561,564,madimba,50.17,20.99,37.36,39.00,96.0,0.0,5.21,PL,1.604202e+09
562,565,nioro,61.77,31.02,33.94,33.94,91.0,0.0,3.56,RU,1.604202e+09
563,566,baturaja,-16.81,26.95,63.79,63.79,65.0,0.0,7.90,ZM,1.604202e+09
564,567,adrar,-19.80,-174.35,77.00,77.00,83.0,90.0,18.34,TO,1.604202e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_data_df[["City Latitude", "City Longitude"]].astype(float)
humidity = cities_data_df["Humidity (%)"].astype(float)

In [5]:
fig = gmaps.figure(center = (-2.77, -42.03), zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cities_weather = cities_data_df.loc[(cities_data_df["Max Temperature (F)"]<90) & (cities_data_df["Max Temperature (F)"]>70)
                                       & (cities_data_df["Wind Speed (mph)"]<10) & (cities_data_df["Cloud Coverage (%)"]<20)
                                       & (cities_data_df['Humidity (%)']<80)]
cities_weather = cities_weather.drop(cities_weather.columns[0], axis = 1)

In [7]:
cities_weather.reset_index(inplace = False)
cities_weather = cities_weather.dropna()
cities_weather.reset_index(drop = True)

,City,City Latitude,City Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),City Country,City Datetime
0,port blair,11.67,92.75,84.07,84.07,74.0,4.0,8.28,IN,1.604202e+09
1,rikitea,-23.12,-134.97,75.90,75.90,72.0,0.0,9.44,PF,1.604202e+09
2,clyde river,-23.70,133.88,87.80,87.80,13.0,0.0,8.05,AU,1.604202e+09
3,vaitupu,15.85,-97.07,78.87,78.87,74.0,10.0,4.21,MX,1.604202e+09
4,bluff,28.65,70.66,83.16,83.16,13.0,0.0,5.37,PK,1.604202e+09
5,sheridan,-35.10,173.78,72.81,72.81,56.0,5.0,9.01,NZ,1.604202e+09
6,nikolskoye,-8.93,125.41,79.66,79.66,40.0,2.0,2.86,TL,1.604202e+09
7,airai,-7.95,110.28,85.15,85.15,66.0,13.0,6.20,ID,1.604202e+09
8,kamenka,29.98,75.38,74.95,74.95,18.0,0.0,5.03,IN,1.604202e+09
9,saleaula,21.44,-157.84,78.48,82.40,61.0,1.0,9.17,US,1.604202e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = cities_weather.reset_index(drop = True)
hotel_df['Hotel Name'] = ""
hotel_df

,City,City Latitude,City Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),City Country,City Datetime,Hotel Name
0,port blair,11.67,92.75,84.07,84.07,74.0,4.0,8.28,IN,1.604202e+09,
1,rikitea,-23.12,-134.97,75.90,75.90,72.0,0.0,9.44,PF,1.604202e+09,
2,clyde river,-23.70,133.88,87.80,87.80,13.0,0.0,8.05,AU,1.604202e+09,
3,vaitupu,15.85,-97.07,78.87,78.87,74.0,10.0,4.21,MX,1.604202e+09,
4,bluff,28.65,70.66,83.16,83.16,13.0,0.0,5.37,PK,1.604202e+09,
5,sheridan,-35.10,173.78,72.81,72.81,56.0,5.0,9.01,NZ,1.604202e+09,
6,nikolskoye,-8.93,125.41,79.66,79.66,40.0,2.0,2.86,TL,1.604202e+09,
7,airai,-7.95,110.28,85.15,85.15,66.0,13.0,6.20,ID,1.604202e+09,
8,kamenka,29.98,75.38,74.95,74.95,18.0,0.0,5.03,IN,1.604202e+09,
9,saleaula,21.44,-157.84,78.48,82.40,61.0,1.0,9.17,US,1.604202e+09,


In [9]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    #define variables for latitude and longitude
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    
    params = {"location": f"{lat},{lng}", "rankby": "distance", "type": "lodging", "key": g_key,}
    city_name = row["City"]

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No hotel nearby... Skipping.")
        print("------------")

Retrieving Results for Index 0: port blair.
Closest hotel to port blair is SRI NITHI TRAVELS (Andaman).
Retrieving Results for Index 1: rikitea.
Closest hotel to rikitea is Pension Maro'i.
Retrieving Results for Index 2: clyde river.
Closest hotel to clyde river is Elkira Court Motel.
Retrieving Results for Index 3: vaitupu.
Closest hotel to vaitupu is Casa Kathleen.
Retrieving Results for Index 4: bluff.
Closest hotel to bluff is Noor Mahal fresh ice cream.
Retrieving Results for Index 5: sheridan.
Closest hotel to sheridan is Sabbi's Backpackers.
Retrieving Results for Index 6: nikolskoye.
Closest hotel to nikolskoye is Juvinal Martin.
Retrieving Results for Index 7: airai.
Closest hotel to airai is Tinouyehbro.
Retrieving Results for Index 8: kamenka.
Closest hotel to kamenka is Grand Star Hotel.
Retrieving Results for Index 9: saleaula.
Closest hotel to saleaula is Judith and Paul Ferreira.
Retrieving Results for Index 10: caravelas.
Closest hotel to caravelas is D-26/2, Dolphin Hi

In [10]:
hotel_df

,City,City Latitude,City Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),City Country,City Datetime,Hotel Name
0,port blair,11.67,92.75,84.07,84.07,74.0,4.0,8.28,IN,1.604202e+09,SRI NITHI TRAVELS (Andaman)
1,rikitea,-23.12,-134.97,75.90,75.90,72.0,0.0,9.44,PF,1.604202e+09,Pension Maro'i
2,clyde river,-23.70,133.88,87.80,87.80,13.0,0.0,8.05,AU,1.604202e+09,Elkira Court Motel
3,vaitupu,15.85,-97.07,78.87,78.87,74.0,10.0,4.21,MX,1.604202e+09,Casa Kathleen
4,bluff,28.65,70.66,83.16,83.16,13.0,0.0,5.37,PK,1.604202e+09,Noor Mahal fresh ice cream
5,sheridan,-35.10,173.78,72.81,72.81,56.0,5.0,9.01,NZ,1.604202e+09,Sabbi's Backpackers
6,nikolskoye,-8.93,125.41,79.66,79.66,40.0,2.0,2.86,TL,1.604202e+09,Juvinal Martin
7,airai,-7.95,110.28,85.15,85.15,66.0,13.0,6.20,ID,1.604202e+09,Tinouyehbro
8,kamenka,29.98,75.38,74.95,74.95,18.0,0.0,5.03,IN,1.604202e+09,Grand Star Hotel
9,saleaula,21.44,-157.84,78.48,82.40,61.0,1.0,9.17,US,1.604202e+09,Judith and Paul Ferreira


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>City Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [12]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

# Display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))